In [1]:
import MathOptInterface
using Profile, ProfileView, Dates
using LinearAlgebra
using JuMP, Gurobi, Test, Ipopt, Juniper, Suppressor, Pandas
const GT = MOI.GreaterThan{Float64}
const LT = MOI.LessThan{Float64}

MathOptInterface.LessThan{Float64}

In [2]:
model = Model(Gurobi.Optimizer)

@variable(model, x, Bin)
@variable(model, y, Bin)
@constraint(model, x + y <= 1.5)
@objective(model, Max, x + y)

@time optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0xc5bc6d97
Variable types: 0 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 1.0000000
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%

User-callback call

In [3]:
model = direct_model(Gurobi.Optimizer())
@variable(model, 0 <= x <= 1.0, Int)
@variable(model, 0 <= y <= 1.0, Int)
@objective(model, Max, x + y)
function my_callback_function(cb_data, cb_where::Cint)
    # You can query a callback attribute using GRBcbget
    if cb_where == GRB_CB_MIPNODE
        Gurobi.load_callback_variable_primal(cb_data, cb_where)
        x_val = callback_value(cb_data, x)
        y_val = callback_value(cb_data, y)
        con = @build_constraint(x + y <= 1)
        MOI.submit(model, MOI.UserCut(cb_data), con)
    end
    # Before querying `callback_value`, you must call:
    #

    return
end

MOI.set(model, Gurobi.CallbackFunction(), my_callback_function)
@time optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x12a95a66
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 2.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%

User-callback calls 24, time in user-callback 0.04 sec
  0.043502 seconds (264.36 k allocations: 17.748 MiB, 99.65% 

# Exact Verify

In [2]:
include("../CayleyVerify.jl")
include("../DeepPoly.jl")
import Pickle

# utility functions
function dorefa_to_staircase(k::Int)
    n = 2^k - 1
    slopes = zeros(n+1)
    breakpoints = [-Inf]
    for i in 1:n
        push!(breakpoints, (2*i-1)/n - 1)
    end
    push!(breakpoints, Inf)
    
    constant_terms = [-1.0]
    for i in 1:n
        push!(constant_terms, -1.0 + 2*i/n)
    end
    return StaircaseFunction(breakpoints, slopes, constant_terms)
end

function predict(neural_net, img)
    num_layers = length(neural_net.weights)
    a = img'
    for i in 1:num_layers
        a = a * neural_net.weights[i] + neural_net.biases[i]'
        if i <= num_layers -1
            a = [eval(neural_net.activation[i], a[j]) for j in 1:length(a)]'
        end
    end
    output = a'
    return findmax(output)[2]
end

predict (generic function with 1 method)

In [3]:
net_from_pickle = Pickle.load(open("../models/MNIST-DoReFa3_Dense256-Dense256.pkl"))
f = dorefa_to_staircase(3)
activation = [f, f]
neural_net = NeuralNetwork(net_from_pickle, activation)
print("net loaded")

net loaded

In [4]:
raw_imgs = Pickle.load(open("../imgs/MNIST_images-for-verification"))
imgs = []
for img in raw_imgs
    img = vcat([w' for w in img] ...)
    img = vcat(img'...)
    push!(imgs, img)
end
labels = Pickle.load(open("../imgs/MNIST_labels-for-verification"))
labels = [l+1 for l in labels]
print("images loaded")

images loaded

In [5]:
# play around with smaller epsilons
t = @elapsed mip, variable_neuron_dict, neuron_integervar_dict = init_mip_deeppoly(neural_net, imgs[1], 0.05)
true_label = labels[1]
target_label = 2
set_attribute(mip, "output_flag", true)
last_layer = last(neural_net.weights)
objective = zeros(10)
objective[target_label] = 1.0
objective[true_label] = -1.0
c = last_layer * objective

for z in mip[:z]
    set_binary(z)
end

num_layers = length(neural_net.weights)
final_dim, output_dim = size(last_layer)
@objective(mip, Max, sum(c[i]*mip[:x][num_layers, i] for i in 1:final_dim))
#@time optimize!(mip)

1Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15
Set parameter OutputFlag to value 1


-0.0022692241764161736 x[3,1] - 0.0003622803487814963 x[3,2] + 0.0013901707425247878 x[3,3] - 0.3418678643065505 x[3,4] + 0.0005246367290965281 x[3,5] - 0.0010369061201345176 x[3,6] + 0.000713563155841257 x[3,7] - 0.0024401845294050872 x[3,8] + 0.0010575342166703194 x[3,9] + 0.0007439423789037392 x[3,10] - 0.0023106156731955707 x[3,11] + 0.001303841738263145 x[3,12] - 0.000828511469080695 x[3,13] - 0.0013108639395795763 x[3,14] - 0.0011618216813076288 x[3,15] + 0.0007242877181852236 x[3,16] + 0.001301012996918871 x[3,17] - 0.000801704591140151 x[3,18] - 0.0017929214518517256 x[3,19] + 0.001864776451839134 x[3,20] - 0.6483434443362057 x[3,21] + 0.0012033609491481911 x[3,22] + 0.0011818046041298658 x[3,23] - 0.001873847737442702 x[3,24] + 0.0004001157485618023 x[3,25] + 0.0005756226310040802 x[3,26] + 0.0014854815799481003 x[3,27] - 0.0012031871592625976 x[3,28] - 0.0026092840016644914 x[3,29] + 0.0009927909450198058 x[3,30] - [[...196 terms omitted...]] - 0.0007154121485655196 x[3,227] 

## Big M MIP

In [6]:
function run_m(num_of_callbacks_before_cut)
    mip, variable_neuron_dict, neuron_integervar_dict = init_mip_deeppoly(neural_net, imgs[1], 0.04)
    true_label = labels[1]
    target_label = 2
    set_attribute(mip, "output_flag", false)
    set_optimizer_attribute(mip, "TimeLimit", 100) # Set a time limit of 300 seconds
    last_layer = last(neural_net.weights)
    objective = zeros(10) # always 10 classes
    objective[target_label] = 1.0
    objective[true_label] = -1.0
    c = last_layer * objective

    num_layers = length(neural_net.weights)
    final_dim, output_dim = size(last_layer)
    @objective(mip, Max, sum(c[i]*mip[:x][num_layers, i] for i in 1:final_dim))

    neurons_by_layer = [length(bias) for bias in neural_net.biases] #including input & output layer
    pushfirst!(neurons_by_layer, size(neural_net.weights[1])[1])
    pop!(neurons_by_layer)

    for z in mip[:z]
    set_binary(z)
    end

    alphas = []
    for n in neurons_by_layer
    push!(alphas, zeros(n))
    end
    @time optimize!(mip)
    #print("fractional calls: ", fractional_calls[1], "\n")
    #output = split(output," ")[2]
    output = split(output," ")[2]
    #open("log.txt", "a") do file
    #    write(file, "$output\t$num_of_callbacks_before_cut\tbig-m\n")
    #end
    return output
end


run_m (generic function with 1 method)

## Cayley MIP

In [7]:
function run_cayley(num_of_callbacks_before_cut)
    mip, variable_neuron_dict, neuron_integervar_dict = init_mip_deeppoly(neural_net, imgs[1], 0.04)
    true_label = labels[1]
    target_label = 2
    set_optimizer_attribute(mip, "TimeLimit", 100) # Set a time limit of 300 seconds
    set_attribute(mip, "output_flag", false)
    last_layer = last(neural_net.weights)
    objective = zeros(10) # always 10 classes
    objective[target_label] = 1.0
    objective[true_label] = -1.0
    # Calculate 'c' using matrix multiplication for vectorization
    c = last_layer * objective

    # Define the objective function in a more concise manner
    num_layers = length(neural_net.weights)
    final_dim, _ = size(last_layer) # Assuming output_dim is not used elsewhere
    @objective(mip, Max, sum(c[i]*mip[:x][num_layers, i] for i in 1:final_dim))

    # Efficiently compute neurons_by_layer without mutating the array
    neurons_by_layer = [size(neural_net.weights[1])[1]; [length(bias) for bias in neural_net.biases][1:end-1]]

    # Set integer constraint on z variables in a single loop
    foreach(set_integer, mip[:z])

    # Preallocate alphas with the correct dimensions and fill with zeros
    alphas = [zeros(n) for n in neurons_by_layer]

    #allocation1 = []
    #allocation2 = []

    count = [0]
    time = []
    function callback_cut(cb_data)
        count[1] += 1
        if count[1]%num_of_callbacks_before_cut == 0
            x_val = callback_value.(Ref(cb_data), mip[:x])
            z_val = callback_value.(Ref(cb_data), mip[:z])
            for i in 1:num_layers-1
                bias = neural_net.biases[i]
                weight = neural_net.weights[i]
                n, m = size(weight)
                for j in 1:m
                    num_pieces = neuron_integervar_dict[(i+1,j)]
                    z = [z_val[i+1,j,k] for k in 1:num_pieces]
                    fractional = any(1e-6 < val < 1-1e-6 for val in z)
                    if !fractional
                        continue
                    end
                    y = x_val[i+1, j]
                    x = [x_val[i, k] for k in 1:n]
                    neuron = variable_neuron_dict[mip[:x][i+1, j]]
                    update = update_alpha!(neuron, x, y, z, alphas[i])
                    if update
                        upper_z = generate_zcoef_from_alpha(neuron, alphas[i], GT(y))
                        lower_z = generate_zcoef_from_alpha(neuron, alphas[i], LT(y))
                        upper_con = @build_constraint(mip[:x][i+1, j] <= sum(mip[:x][i, k]*alphas[i][k] for 
                                    k in 1:n) + sum(mip[:z][i+1, j, p]*upper_z[p] for p in 1:num_pieces))
                        lower_con = @build_constraint(mip[:x][i+1, j] >= sum(mip[:x][i, k]*alphas[i][k] for 
                                    k in 1:n) + sum(mip[:z][i+1, j, p]*lower_z[p] for p in 1:num_pieces))
                        MOI.submit(mip, MOI.UserCut(cb_data), upper_con)
                        MOI.submit(mip, MOI.UserCut(cb_data), lower_con)
                        #push!(allocation2, t2+t3)
                    end
                    #push!(time, [time_update, time_uzcoef, time_lzcoef])
                end
            end
        end
    end
    #record opt gap and nodes using verbose model output
    MOI.set(mip, MOI.UserCutCallback(), callback_cut)
    @time optimize!(mip)
    #print("fractional calls: ", fractional_calls[1], "\n")
    output = split(output," ")[2]
    #open("log.txt", "a") do f
    #    write(f, "$output\t$num_of_callbacks_before_cut\tcayley\n")
    #end
    return output
    #return time
end


run_cayley (generic function with 1 method)

In [8]:
profile_time = 0
run_cayley(150)
run_m(150)



1Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [ ]:
print(profile_time)

In [ ]:

val_arrs = [[],[]]
@suppress begin 
    idx = 0
    while idx < 5
        push!(val_arrs[1], parse(Float64, run_cayley(150)))
        push!(val_arrs[2],  parse(Float64,run_m(150)))
        idx += 1
    end
end
cayley_avg = mean(val_arrs[1])
m_avg = mean(val_arrs[2])
print("$cayley_avg\t$m_avg\n")
#running tally:
# cayley: III
# m: 
# cayley: 14.891829570000004	m: 14.952998299999997 (averaged over 100 runs)



In [ ]:
run_cayley(150)
run_m(150)

In [ ]:
# B
write("log.txt", "time\tcut_freq\tmethod\n")
cayley_results = Dict{Int, Array{Any,1}}()
m_results = Dict{Int, Array{Any,1}}()

for num in range(25, 500, step=25)
    cayley_results[num] = []
    m_results[num] = []
    for _ in range(1,5)
        push!(cayley_results[num], run_cayley(num))
    end
    for _ in range(1,5)
        push!(m_results[num], run_m(num))
    end
end


In [ ]:
avg_results = []
#print("cayley_results: $cayley_results\n")
for (key, value) in cayley_results
    print("key: $key, value: $value\n")
    final_val = [parse(Float64, val) for val in value]
    push!(avg_results, (key, mean(final_val), "cayley"))
end

for (key, value) in m_results
    print("key: $key, value: $value\n")
    final_val = [parse(Float64, val) for val in value]
    push!(avg_results, (key, mean(final_val), "big-m"))
end


In [ ]:
#cayley_callback_debugging(150)
avg_results = sort!(avg_results, by=x->x[2])

print(avg_results)